In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Churn prediction for game developers using Google Analytics 4 and BigQuery ML

<table align="left">

  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/gaming_churn_prediction/churn_prediction_for_game_developers.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/gaming_churn_prediction/churn_prediction_for_game_developers.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/gaming_churn_prediction/churn_prediction_for_game_developers.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Table of contents

* [Overview](#section-1)
* [Objective](#section-2)
* [Dataset](#section-3)
* [Costs](#section-4)
* [Create a BigQuery dataset](#section-5)
* [Explore data](#section-6)
* [Preparing the training data](#section-7)
	* [Identifying the label for each user](#section-7-subsection-1)
    * [Extracting demographic data for each user](#section-7-subsection-2)
	* [Extracting behavioral data for each user](#section-7-subsection-3)
	* [Combining the label, demographic and behavioral data together as training data](#section-7-subsection-4)
* [Training the propensity model with BigQuery ML](#section-8)
* [Model evaluation](#section-9)
	* [Confusion matrix: predicted vs actual values](#section-9-subsection-1)
	* [ROC Curve](#section-9-subsection-2)
* [Model prediction](#section-10)  
* [Export predictions table to Cloud Storage](#section-11)
* [Clean up](#section-12)

## Overview
<a name="section-1"></a>

This tutorial shows you how to train, evaluate a propensity model in BigQuery ML to predict user retention on a mobile game, based on app measurement data from Google Analytics 4.

Learn more about [Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction) and Learn more about [BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml#machine_learning_directly_in).

### Objective
<a name="section-2"></a>

In this tutorial, you learn how to train, evaluate a propensity model in BigQuery ML.

This tutorial uses the following Google Cloud ML services and resources:
* BigQuery.

The steps performed include:

* Explore an export of Google Analytics 4 data on BigQuery.
* Prepare the training data using demographic, behavioral data, and labels (churn/not-churn).
* Train an XGBoost model using BigQuery ML.
* Evaluate a model using BigQuery ML.
* Make predictions on which users will churn using BigQuery ML.

### Dataset
<a name="section-3"></a>

This notebook uses [this public BigQuery dataset](https://console.cloud.google.com/bigquery?p=firebase-public-project&d=analytics_153293282&t=events_20181003&page=table), which contains raw event data from a real mobile gaming app called Flood It! ([Android app](https://play.google.com/store/apps/details?id=com.labpixies.flood), [iOS app](https://itunes.apple.com/us/app/flood-it!/id476943146?mt=8)). The [data schema](https://support.google.com/analytics/answer/7029846) originates from Google Analytics for Firebase, but is the same schema as [Google Analytics 4](https://support.google.com/analytics/answer/9358801); the techniques in this notebook can be applied to either Google Analytics for Firebase or Google Analytics 4 data.

Google Analytics 4 (GA4) uses an [event-based](https://support.google.com/analytics/answer/9322688) measurement model. Events provide insight on what is happening in an app or on a website, such as user actions, system events, or errors. Every row in the dataset is an event, with various characteristics relevant to that event stored in a nested format within the row. While Google Analytics logs many types of events already by default, developers can also customize the types of events they also wish to log.

Note that as you cannot simply use the raw event data to train a machine learning model, this notebook shows you some important steps of how to pre-process the raw data into an appropriate format to use as training data for classification models.

### Costs
<a name="section-4"></a>

This tutorial uses the following billable components of Google Cloud:

* Vertex AI
* BigQuery
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery
pricing](https://cloud.google.com/bigquery/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install additional packages


In [ ]:
! pip3 install --quiet --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]'

### Colab Only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Before you begin

#### Set your project ID

**If you don't know your project ID**, try the following:
-  Run `gcloud config list`
-  Run `gcloud projects list`
-  See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable used by Vertex AI. 
Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench** 
- Do nothing as you are already authenticated.

**2. Local JupyterLab Instance,** uncomment and run.

In [ ]:
# ! gcloud auth login

**3. Colab,** uncomment and run:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service Account or other**
- See all the authentication options here: [Google Cloud Platform Jupyter Notebook Authentication Guide](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_NAME = "your-bucket-name-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Import libraries and define constants

In [ ]:
import os

import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud.bigquery import Client

#### Initialize BigQuery Client

In [ ]:
client = Client(project=PROJECT_ID)

## Create a BigQuery dataset
<a name="section-5"></a>


If you are using ***Vertex AI Workbench managed notebooks instance***, every cell which starts with "#@bigquery" will be a SQL Query. If you are using Vertex AI Workbench user managed notebooks instance or Colab it will be a markdown cell.

In this notebook, you create a dataset in your project. To create it, run the following cell:

#@bigquery
-- create a dataset in BigQuery

CREATE SCHEMA bqmlga4
OPTIONS(
  location="us"
  )

(**Optional**)If you are using Vertex AI Workbench managed notebooks instance, once the results from BigQuery are displayed in the above cell, click the **Query and load as DataFrame** button and execute the generated code stub to fetch the data into the current notebook as a dataframe.

*Note: By default the data is loaded into a `df` variable, though this can be changed before executing the cell if required.*

In [ ]:
dataset_id = "bqmlga4" + "_" + UUID

In [ ]:
query = """
CREATE SCHEMA `{PROJECT_ID}.{dataset_id}`
OPTIONS(
  location="us"
  )
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id
)
query_job = client.query(query)

print(query_job.result())

## Explore data
<a name="section-6"></a>


The sample dataset contains raw event data, as shown in the next cell:


In [ ]:
query = """
SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`
    
LIMIT 5
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

It may be helpful to take a look at the overall schema used in Google Analytics 4. As mentioned earlier, Google Analytics 4 uses an event-based measurement model and each row in this dataset is an event. [View the complete schema and details about each column](https://support.google.com/analytics/answer/7029846). As you can see above, specific columns are nested records and contain detailed information:

* `app_info`
* `device`
* `ecommerce`
* `event_params`
* `geo`
* `traffic_source`
* `user_properties`
* `items` (present by default in GA4 datasets)
* `web_info` (present by default in GA4 datasets)

The query results below show that there are 15K users and 5.7M events in this dataset:

#@bigquery
SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`

In [ ]:
query = """
SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

## Preparing the training data
<a name="section-7"></a>


You cannot simply use raw event data to train a machine learning model as it would not be in the right shape and format to use as training data. So in this section, you learn how to pre-process the raw data into an appropriate format to use as training data for classification models.


To predict which user is going to _churn_ or _return_, the ideal training data format for classification should look like the following:  


|User ID|User demographic data|User behavioral data|Churned|
|-|-|-|-|
|User1|(e.g., country, device_type)|(e.g., # of times they did something within a time period)|1
|User2|(e.g., country, device_type)|(e.g., # of times they did something within a time period)|0
|User3|(e.g., country, device_type)|(e.g., # of times they did something within a time period)|1


Characteristics of the training data:
- Each row is a separate unique user ID
- Feature(s) for **demographic data**
- Feature(s) for **behavioral data**
- The actual **label** that you want to train the model to predict (for example, 1 = churned, 0 = returned)

You can train a model with only demographic data or behavioral data, but having a combination of both likely help you create a more predictive model. For this reason, in this section, you learn how to pre-process the raw data to follow this training data format.

The following sections walk you through preparing the demographic data, behavioral data, and the label before joining them all together as the complete training dataset. The steps are:

1. Identify the label for each user (churned or returned)
1. Extract demographic data for each user
1. Extract behavioral data for each user
1. Combine the label, demographic data, and behavioral data together as training data

#### Step 1: Identify the label for each user
<a name="section-7-subsection-1"></a>

The raw dataset doesn't have a feature that simply identifies users as "churned" or "returned", so in this section, you need to create this label based on some of the existing columns.

There are many ways to define user churn, but for the purposes of this notebook, you predict 1-day churn as users who do not come back and use the app again after 24 hours of the user's first engagement. 

In other words, after 24 hours of a user's first engagement with the app:
- if the user _shows no event data thereafter_, the user is considered **churned**. 
- if the user _does have at least one event datapoint thereafter_, then the user is considered **returned**

You may also want to remove users who were unlikely to have ever returned anyway after spending just a few minutes with the app, which is sometimes referred to as "bouncing". For example, you may want to build the model only on users who spent at least 10 minutes with the app (users who did not bounce).

So your updated definition of a **churned user** for this notebook is:
> "any user who spent at least 10 minutes on the app, but after 24 hours from when they first engaged with the app, never used the app again"


In SQL, since the raw data contains all of the events for every user, from their first touch (app installation) to their last touch, you can use this information to create two columns: `churned` and `bounced`.


Take a look at the following SQL query and the results:

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  bqmlga4.returningusers 
LIMIT 100;

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  {dataset_id}.returningusers 
LIMIT 100;
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#@bigquery
SELECT 
  * 
FROM 
  bqmlga4.returningusers 

In [ ]:
query = """
SELECT 
  * 
FROM 
  {dataset_id}.returningusers 
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

For the `churned` column, `churned = 0` if the user performs an action after 24 hours since their first touch, otherwise if their last action was only within the first 24 hours, then `churned = 1`.


For the `bounced` column, `bounced = 1` if the user's last action was within the first ten minutes since their first touch with the app, otherwise `bounced = 0`. You can use this column to filter the training data later on by conditionally querying for users where `bounced = 0`.

You might wonder how many of these 15k users bounced and returned? You can run the following query to check:

#@bigquery
SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    bqmlga4.returningusers
GROUP BY 1,2
ORDER BY bounced

In [ ]:
query = """
SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    {dataset_id}.returningusers
GROUP BY 1,2
ORDER BY bounced
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

For the training data, you only end up using data where `bounced = 0`. Based on the 15k users, you can see that 5,557 (\~41%) users bounced within the first ten minutes of their first engagement with the app, but of the remaining 8,031 users, 1,883 users (\~23%) churned after 24 hours.

#@bigquery
SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    bqmlga4.returningusers
WHERE bounced = 0

In [ ]:
query = """
SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    {dataset_id}.returningusers
WHERE bounced = 0
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

There are 23% churners in the data which is not bad for training a churn prediction model. If the class imbalance seems to be high, oversampling or undersampling techniques can be considered to balance the class distribution.

#### Step 2: Extract demographic data for each user
<a name="section-7-subsection-2"></a>


This section is focused on extracting the demographic information for each user. Different demographic information about the user is available in the dataset already, including `app_info`, `device`, `ecommerce`, `event_params`, and `geo`. Demographic features can help the model predict whether users on certain devices or countries are more likely to churn.

For this notebook, you can start just with `geo.country`, `device.operating_system`, and `device.language`. If you are using your own dataset and have joinable first-party data, this section is a good opportunity to add any additional attributes for each user that may not be readily available in Google Analytics 4.

Note that a user's demographics may occasionally change (e.g. moving from one country to another). For simplicity, you just use the demographic information that Google Analytics 4 provides when the user LAST engaged with the app as indicated by `MAX(event_timestamp)`. This enables every unique user to be represented by a single row.

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  bqmlga4.user_demographics
LIMIT 10

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  {dataset_id}.user_demographics
LIMIT 10
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#@bigquery
SELECT
  *
FROM
  bqmlga4.user_demographics


In [ ]:
query = """
SELECT
  *
FROM
  {dataset_id}.user_demographics
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#### Step 3: Extract behavioral data for each user
<a name="section-7-subsection-3"></a>

Behavioral data in the raw event data spans across multiple events -- and thus rows -- per user. The goal of this section is to aggregate and extract behavioral data for each user, resulting in one row of behavioral data per unique user.

But what kind of behavioral data you need to prepare? Since the end goal of this notebook is to predict, based on a user's activity within the first 24 hours since app installation, whether that user will churn or return thereafter, then you want to use behavioral data from the first 24 hours in your training data. Later on, you can also extract some extra time-related features from `user_first_engagement`, such as the month or day of the first engagement.

Google Analytics automatically collects [specific events](https://support.google.com/analytics/answer/6317485) that you can use to analyze behavior. In addition, there are recommended [events for games](https://support.google.com/analytics/answer/6317494). 


As a first step, you can explore all the unique events that exist in this dataset, based on `event_name`:

#@bigquery
SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC

In [ ]:
query = """
SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

For this tutorial, to predict whether a user will churn or return, you can start by counting the number of times a user engages in the following event types:

* `user_engagement`
* `level_start_quickplay`
* `level_end_quickplay`
* `level_complete_quickplay`
* `level_reset_quickplay`
* `post_score`
* `spend_virtual_currency`
* `ad_reward`
* `challenge_a_friend`
* `completed_5_levels`
* `use_extra_steps`

In SQL, you can aggregate the behavioral data by calculating the total number of times when each of the above `event_names` occurred in the dataset per user.

If you are using your own dataset, you may have different event types that you can aggregate and extract. Your app may be sending very different `event_names` to Google Analytics so be sure to use events suitable to your scenario.

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      bqmlga4.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  bqmlga4.user_aggregate_behavior
LIMIT 10


In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      {dataset_id}.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  {dataset_id}.user_aggregate_behavior
LIMIT 10

""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#### Step 4: Combine the label, demographic data, and behavioral data together as training data
<a name="section-7-subsection-4"></a>

In this section, you can now combine these three intermediary views (label, demographic data, and behavioral data) into the final training dataset. Here you can also specify `bounced = 0`, in order to limit the training data only to users who did not "bounce" within the first 10 minutes of using the app.

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    bqmlga4.returningusers ret
  LEFT OUTER JOIN
    bqmlga4.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    bqmlga4.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  bqmlga4.train
LIMIT 10

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    {dataset_id}.returningusers ret
  LEFT OUTER JOIN
    {dataset_id}.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    {dataset_id}.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  {dataset_id}.train
LIMIT 10
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#@bigquery
SELECT
  *
FROM
  bqmlga4.train

In [ ]:
query = """
SELECT
  *
FROM
  {dataset_id}.train
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
df = query_job.to_dataframe()

In [ ]:
df

Check the percentage of null values.

In [ ]:
(
    100 * df[["operating_system", "language", "country"]].isna().sum() / df.shape[0]
).plot.bar(figsize=(15, 4))
plt.title("Null-percentage of the columns")
plt.show()

## Training the propensity model with BigQuery ML
<a name="section-8"></a>


In this section, using the training data you prepared, you now train machine learning models in SQL using BigQuery ML.

You use an [XGBoost](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree) model here. In this notebook, the model predicts whether the user will churn (1) or return (0) after 24 hours of the user's first engagement with the app.


#### Train an XGBoost model

The following code trains an XGBoost model. This may take several minutes.

For more information on the default hyperparameters used, see [The CREATE MODEL statement for boosted tree models using XGBoost](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree).

#@bigquery
CREATE OR REPLACE MODEL bqmlga4.churn_xgb

OPTIONS(
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  DATA_SPLIT_METHOD='RANDOM',
  DATA_SPLIT_EVAL_FRACTION=0.2,
    
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  *
FROM
  bqmlga4.train

In [ ]:
query = """
CREATE OR REPLACE MODEL {dataset_id}.churn_xgb

OPTIONS(
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  DATA_SPLIT_METHOD='RANDOM',
  DATA_SPLIT_EVAL_FRACTION=0.2,
    
  INPUT_LABEL_COLS=["churned"]
) AS

SELECT
  *
FROM
  {dataset_id}.train
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.result()

## Model evaluation
<a name="section-9"></a>


To evaluate the model, you can run [`ML.EVALUATE`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) on a model that has finished training to inspect some of the metrics.

The metrics are based on the test sample data that was automatically split during model creation ([see the CREATE MODEL documentation for more information](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method)).

#@bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_xgb)


In [ ]:
query = """
SELECT
  *
FROM
  ML.EVALUATE(MODEL {dataset_id}.churn_xgb)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

`ML.EVALUATE` generates the `precision`, `recall`, `accuracy`, and `f1_score` using the default classification threshold of 0.5, which can be modified by using the optional [`THRESHOLD`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#eval_threshold) parameter.

Generally speaking, you can use the `log_loss` and `roc_auc` metrics to compare  model performance.

The `log_loss` ranges between 0 and 1.0, and the closer the `log_loss` is to zero, the closer the predicted labels were to the actual labels.

The `roc_auc` ranges between 0 and 1.0, and the closer the `roc_auc` is to 1.0, the better the model is at distinguishing between the classes.

For more information on these metrics, you can read through the definitions on [precision and recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall), [accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy), [f1-score](https://en.wikipedia.org/wiki/F-score), [log_loss](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss) and [roc_auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc).

#### Confusion matrix: predicted vs actual values
<a name="section-9-subsection-1"></a>


In addition to model evaluation metrics, you may also want to use a confusion matrix to inspect how well the model predicted the labels, compared to the actual labels.

With the rows indicating the actual labels, and the columns as the predicted labels, the resulting format for ML.CONFUSION_MATRIX for binary classification looks like:

| | Predicted_0 | Predicted_1|
|-|-|-|
|Actual_0| True Negatives | False Positives|
|Actual_1| False Negatives | True Positives|

For more information on confusion matrices, see [Classification: True vs. False and Positive vs. Negative](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative).

#@bigquery
SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL bqmlga4.churn_xgb)

In [ ]:
query = """
SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL {dataset_id}.churn_xgb)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

#### ROC Curve
<a name="section-9-subsection-2"></a>


#@bigquery
SELECT * FROM ML.ROC_CURVE(MODEL bqmlga4.churn_xgb)

In [ ]:
query = """
SELECT * FROM ML.ROC_CURVE(MODEL {dataset_id}.churn_xgb)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

## Model prediction
<a name="section-10"></a>


You can run [`ML.PREDICT`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict) to make predictions on the propensity to churn. The following code returns all of the information from `ML.PREDICT`.

#@bigquery
CREATE OR REPLACE VIEW bqmlga4.prediction_data AS(
(SELECT * FROM bqmlga4.train where churned=1 limit 10)
union all
(SELECT * FROM bqmlga4.train where churned=0 limit 20))

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.prediction_data AS(
(SELECT * FROM {dataset_id}.train where churned=1 limit 10)
union all
(SELECT * FROM {dataset_id}.train where churned=0 limit 20))
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.result()

#@bigquery
SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.prediction_data)
            ) 

In [ ]:
query = """
SELECT
  *
FROM
  ML.PREDICT(MODEL {dataset_id}.churn_xgb,
  (SELECT * FROM {dataset_id}.prediction_data)
            ) 
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

For propensity modeling, the most important output is the probability of a behavior occuring. The following query returns the probability that the user will return after 24 hours. The higher the probability and closer it is to 1, the more likely the user is predicted to churn, and the closer it is to 0, the more likely the user is predicted to return.

#@bigquery
SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.train))

In [ ]:
query = """
SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL {dataset_id}.churn_xgb,
  (SELECT * FROM {dataset_id}.train))
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

## Export the predictions table to Cloud Storage
<a name="section-11"></a>

There are several ways to export the predictions table to Cloud Storage, so that you can use the information in a separate service. Perhaps the easiest way is to export directly to Cloud Storage using SQL. [Learn more about the EXPORT DATA statement](https://cloud.google.com/bigquery/docs/reference/standard-sql/other-statements#export_data_statement).

In [ ]:
query = """
CREATE OR REPLACE TABLE {dataset_id}.prediction_data_table AS (
SELECT 
  * 
FROM 
  {dataset_id}.prediction_data
)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.result()

In [ ]:
FILE_PATH = BUCKET_URI + "/" + "*.csv"

In [ ]:
query = """
EXPORT DATA OPTIONS (
uri= @FILE_PATH, 
  format=CSV,
  header=True, 
  overwrite=True 
    
) AS 
SELECT
  * from {dataset_id}.prediction_data_table
""".format(
    dataset_id=dataset_id
)

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("FILE_PATH", "STRING", FILE_PATH),
    ]
)
query_job = client.query(query, job_config=job_config)  # Make an API request.

In [ ]:
query_job.result()

## Clean up
<a name="section-12"></a>

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial. The following code deletes the entire dataset.

In [ ]:
# Set dataset variable to the ID of the BigQuery dataset to fetch.
dataset = f"{PROJECT_ID}.{dataset_id}"

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    dataset, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(dataset_id))

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI